## Instalações

In [1]:
pip install gdown

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

## Importação dos dados

Os dados de consumo de água e fraudes foram obtidos de arquivos CSV e importados diretamente para o Google Colab. O Google Drive foi utilizado como repositório para esses arquivos, facilitando o acesso e a manipulação dos dados.

In [6]:
!gdown 1VqnuZFz0bytf5kPoo4wS5kRTsIwJZGC4

Downloading...
From (original): https://drive.google.com/uc?id=1VqnuZFz0bytf5kPoo4wS5kRTsIwJZGC4
From (redirected): https://drive.google.com/uc?id=1VqnuZFz0bytf5kPoo4wS5kRTsIwJZGC4&confirm=t&uuid=ef63f500-bcb1-4fa9-98ad-a92d956fdc1a
To: /content/CONSUMO_2023.zip
100% 221M/221M [00:01<00:00, 139MB/s]


In [7]:
!gdown 1X8B08S8HomStfeVsLHfxgGQIq1Xugu1D

Downloading...
From: https://drive.google.com/uc?id=1X8B08S8HomStfeVsLHfxgGQIq1Xugu1D
To: /content/FRAUDES_HIST.zip
100% 19.5M/19.5M [00:00<00:00, 26.0MB/s]


In [ ]:
consumo_2023 = pd.read_csv('/content/CONSUMO_2023.zip', delimiter=';')
fraudes = pd.read_csv('/content/FRAUDES_HIST.zip', delimiter=';')

## Análise exploratória dos dados de fraudes e consumo

Nesta seção, realizamos uma análise exploratória inicial para entender melhor os dados de consumo e fraudes. A análise incluiu a verificação da estrutura dos dados, identificação de valores únicos e o tratamento de valores ausentes.

## Dados de consumo


1. **Número de registros e variáveis**: Analisamos o número de linhas e colunas para entender a dimensão dos dados.

2. **Exceções e ocorrências:** Investigamos valores únicos em colunas específicas como 'EXCECAO', 'DSC_SIMULTANEA', 'CATEGORIA' e 'DSC_OCORRENCIA' para identificar padrões ou anomalias.

3. **Tipos de dados e valores nulos**: Verificamos os tipos de dados e a presença de valores nulos, que são essenciais para preparar os dados para a análise.

In [ ]:
consumo_2023.shape

In [ ]:
consumo_2023.dtypes

In [ ]:
consumo_2023.isnull().sum()

In [ ]:
consumo_2023.describe()

In [ ]:
consumo_2023.info()

In [ ]:
consumo_2023.columns

In [ ]:
consumo_2023['EXCECAO'].unique()
# se a pessoa é isenta de agua e esgoto, há a possibilidade de estar fraudando algo? O que leva alguém a ser isento de agua e esgoto?

In [ ]:
consumo_2023['DSC_SIMULTANEA'].unique()
# o que leva uma conta de água ou esgoto a ser retida? O que caracteriza essa definição?

In [ ]:
consumo_2023['CATEGORIA'].unique()

In [ ]:
consumo_2023['DSC_OCORRENCIA'].unique()

## Dados de fraudes

**1. Estrutura dos dados:** Análise do formato dos dados de fraudes, incluindo a verificação do número de registros, tipos de dados e valores nulos.

**2. Resumo estatístico:** Geração de estatísticas descritivas para entender melhor a distribuição dos dados.

In [ ]:
fraudes.shape

In [ ]:
fraudes.head()

In [ ]:
fraudes.dtypes

In [ ]:
fraudes.describe()

In [ ]:
fraudes.info()

In [ ]:
fraudes.columns

## Dados de consumo atrelados aos dados de fraudes

Para integrar os dados de consumo e fraudes, realizamos as seguintes etapas:

**Filtragem de registros de 2023:** Selecionamos apenas os registros referentes ao ano de 2023 nos dados de fraudes.

**Conversão de datas:** As colunas de datas foram convertidas para o formato datetime para facilitar as operações temporais.

**Unificação dos dados:** Os dados de consumo e fraudes foram unidos usando as colunas MATRICULA e ANO_MES, e uma nova coluna fraude foi criada para indicar se uma fraude ocorreu ou não.

In [ ]:
# Filtrar o DataFrame 'fraudes' para conter apenas registros de 2023
fraudes_2023 = fraudes[fraudes['ANOMES'].str.endswith('23')]

# Converter a coluna 'REFERENCIA' para datetime no DataFrame de consumo
consumo_2023['REFERENCIA'] = pd.to_datetime(consumo_2023['REFERENCIA'], errors='coerce')

# Converter a coluna 'ANOMES' do DataFrame de fraudes para datetime
fraudes_2023['ANOMES'] = pd.to_datetime(fraudes_2023['ANOMES'], format='%m/%y', errors='coerce')

# Criar a coluna 'ANO_MES' no DataFrame de consumo e fraudes para unificação
consumo_2023['ANO_MES'] = consumo_2023['REFERENCIA'].dt.to_period('M').astype(str)
fraudes_2023['ANO_MES'] = fraudes_2023['ANOMES'].dt.to_period('M').astype(str)

# Realizar o merge entre o DataFrame de consumo e o de fraudes, utilizando as colunas 'MATRICULA' e 'ANO_MES'
fraudes_consumo_2023 = consumo_2023.merge(fraudes_2023[['MATRICULA', 'ANO_MES']],
                                          how='left',
                                          on=['MATRICULA', 'ANO_MES'],
                                          indicator=True)

# Criar a coluna 'fraude' com base no merge:
fraudes_consumo_2023['fraude'] = fraudes_consumo_2023['_merge'].apply(lambda x: 1 if x == 'both' else 0)

# Remover a coluna '_merge' que foi criada automaticamente pelo merge
fraudes_consumo_2023.drop(columns=['_merge'], inplace=True)


### Calculo da fatura mensal otimizada

Para calcular a fatura mensal de cada matrícula, utilizamos uma função que considera as tarifas fixas e variáveis de acordo com a subcategoria de consumo. Essa função foi aplicada apenas às linhas onde fraudes foram confirmadas, a fim de diminuir o gasto de memória RAM do colab.

In [ ]:
tarifas_data = {
    'SUB_CATEGORIA': ['RES. SOCIAL', 'RESIDENCIAL', 'COMERCIAL', 'INDUSTRIAL', 'PUBLICA', 'Poder Público Municipal', 'RES. INDUBRASIL'],
    'TARIFA_FIXA': [8.07, 17.78, 26.98, 42.40, 90.00, 90.00, 17.47],
    'FAIXA_CONSUMO_FIXA': [20, 10, 10, 10, 20, 20, 10],  # Faixa fixa antes de aplicar tarifas variáveis
    'TARIFA_AGUA': [
        [(20, 3.26, 2.28)],  # RES. SOCIAL (até 20m³)
        [(10, 7.19, 5.03), (15, 9.20, 6.44), (20, 9.36, 6.55), (25, 10.35, 7.25),
         (30, 12.73, 8.91), (50, 15.23, 10.66), (float('inf'), 16.79, 11.75)],  # RESIDENCIAL
        [(10, 9.92, 6.94), (float('inf'), 20.31, 14.22)],  # COMERCIAL
        [(10, 15.49, 10.84), (float('inf'), 29.83, 20.88)],  # INDUSTRIAL
        [(20, 9.01, 6.31), (float('inf'), 37.39, 26.17)],  # PUBLICA
        [(20, 7.24, 5.07), (float('inf'), 9.74, 6.82)],  # Poder Público Municipal
        [(float('inf'), 2.54, 1.75)]  # RES. INDUBRASIL
    ]
}

# Criar o DataFrame de tarifas
df_tarifas = pd.DataFrame(tarifas_data)

# Função otimizada para calcular a fatura mensal
def calcular_fatura_otimizada_2023(row):
    subcategoria = row['SUB_CATEGORIA']
    consumo = row['CONS_MEDIDO']

    # Verificar se a subcategoria está nas tarifas
    if subcategoria not in df_tarifas['SUB_CATEGORIA'].values:
        return None

    # Obter a tarifa correspondente à subcategoria
    tarifa_info = df_tarifas[df_tarifas['SUB_CATEGORIA'] == subcategoria].iloc[0]
    tarifa_fixa = tarifa_info['TARIFA_FIXA']
    faixa_consumo_fixa = tarifa_info['FAIXA_CONSUMO_FIXA']
    faixas = tarifa_info['TARIFA_AGUA']

    # Inicializar a fatura com a tarifa fixa
    fatura_fixa = tarifa_fixa
    consumo_excedente = max(0, consumo - faixa_consumo_fixa)

    # Cálculo do valor variável
    valor_variavel = 0
    for limite, tarifa_agua, tarifa_esgoto in faixas:
        if consumo_excedente > 0:
            if consumo_excedente <= limite:
                valor_variavel += consumo_excedente * (tarifa_agua + tarifa_esgoto)
                consumo_excedente = 0
                break
            else:
                valor_variavel += limite * (tarifa_agua + tarifa_esgoto)
                consumo_excedente -= limite

    # Cálculo final da fatura
    fatura_total = fatura_fixa + valor_variavel
    return fatura_total * 1.0414  # Aplicar reajuste de 4,14%

# Aplicar a função otimizada de cálculo de fatura apenas às linhas onde fraude = 1
fraudes_consumo_2023.loc[fraudes_consumo_2023['fraude'] == 1, 'FATURA_MENSAL'] = fraudes_consumo_2023[
    fraudes_consumo_2023['fraude'] == 1].apply(calcular_fatura_otimizada_2023, axis=1)

fraudes_consumo_2023


### Análises exploratórias adicionais


1. **Quantidade de matrículas únicas e reincidentes:** Verificamos a quantidade total de matrículas e quantas destas reincidiram em fraudes no ano de 2023.

2. **Distribuição das fraudes:** Analisamos a distribuição das fraudes por subcategoria, assim como o valor total e a média de consumo para cada subcategoria.

In [ ]:
import pandas as pd

def analises_exploratorias_2023(fraudes_consumo_2023):
    # Quantidade total de matrículas únicas
    matriculas_unicas_2023 = fraudes_consumo_2023['MATRICULA'].nunique()

    # Quantidade de matrículas reincidentes na fraude
    matriculas_fraudulentas_2023 = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 1]
    matriculas_reincidentes_2023 = matriculas_fraudulentas_2023.groupby('MATRICULA').size()
    matriculas_reincidentes_2023 = matriculas_reincidentes_2023[matriculas_reincidentes_2023 > 1].count()

    # Distribuição das fraudes por subcategoria
    distribuicao_subcategorias_2023 = matriculas_fraudulentas_2023['SUB_CATEGORIA'].value_counts()

    # Valor total de fraudes por subcategoria
    valor_total_fraudes_subcategoria_2023 = matriculas_fraudulentas_2023.groupby('SUB_CATEGORIA')['FATURA_MENSAL'].sum().round(2)

    # Média de consumo por subcategoria nas fraudes
    media_consumo_subcategoria_2023 = matriculas_fraudulentas_2023.groupby('SUB_CATEGORIA')['CONS_MEDIDO'].mean().round(2)

    # Mediana de consumo por subcategoria nas fraudes
    mediana_consumo_subcategoria_2023 = matriculas_fraudulentas_2023.groupby('SUB_CATEGORIA')['CONS_MEDIDO'].median().round(2)

    # Distribuição de fraudes ao longo dos meses
    distribuicao_mensal_2023 = matriculas_fraudulentas_2023['ANO_MES'].value_counts().sort_index()

    return {
        'matriculas_unicas': matriculas_unicas_2023,
        'matriculas_reincidentes': matriculas_reincidentes_2023,
        # 'distribuicao_subcategorias': distribuicao_subcategorias_2023,
        # 'valor_total_fraudes_subcategoria': valor_total_fraudes_subcategoria_2023,
        # 'media_consumo_subcategoria': media_consumo_subcategoria_2023,
        # 'mediana_consumo_subcategoria': mediana_consumo_subcategoria_2023,
        # 'distribuicao_mensal': distribuicao_mensal_2023
    }

resultados_2023 = analises_exploratorias_2023(fraudes_consumo_2023)
resultados_2023

Geramos tabelas que resumem as fraudes por categoria, incluindo o número de fraudes, valor total e valor médio por fraude.

In [ ]:
tabela_valor_total_2023 = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 1].groupby('CATEGORIA')['FATURA_MENSAL'].sum().reset_index()
tabela_valor_total_2023.columns = ['CATEGORIA', 'VALOR_TOTAL_FRAUDES']
tabela_valor_total_2023['VALOR_TOTAL_FRAUDES'] = tabela_valor_total_2023['VALOR_TOTAL_FRAUDES'].round(2)

# Tabela 2: Número de fraudes, valor total e valor médio por fraude dividido por categoria
tabela_resumo_fraudes_2023 = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 1].groupby('CATEGORIA').agg(
    NUMERO_FRAUDES=('fraude', 'count'),
    VALOR_TOTAL_FRAUDES=('FATURA_MENSAL', 'sum'),
    VALOR_MEDIO_FRAUDE=('FATURA_MENSAL', 'mean')
).reset_index()

# Arredondando os valores para duas casas decimais
tabela_resumo_fraudes_2023['VALOR_TOTAL_FRAUDES'] = tabela_resumo_fraudes_2023['VALOR_TOTAL_FRAUDES'].round(2)
tabela_resumo_fraudes_2023['VALOR_MEDIO_FRAUDE'] = tabela_resumo_fraudes_2023['VALOR_MEDIO_FRAUDE'].round(2)

tabela_resumo_fraudes_2023


Contagem da quantidade de fraudes

In [ ]:
fraudes_consumo_2023['fraude'].value_counts()

Percentual de fraudes

In [ ]:
percentual_de_fraudes=(fraudes_consumo_2023['fraude'].value_counts()[1]/fraudes_consumo_2023['fraude'].value_counts()[0])*100
percentual_de_fraudes

In [ ]:
fraudes_confirmadas = fraudes_consumo_2023[fraudes_consumo_2023['fraude']==1]
fraudes_confirmadas

Fraudes por categoria

In [ ]:
contagem_fraudes_por_categoria = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 1]['CATEGORIA'].value_counts()
contagem_fraudes_por_categoria

Consumo médio das fraudes por categoria

In [ ]:
consumo_medio_por_categoria = fraudes_confirmadas.groupby('CATEGORIA')['CONS_MEDIDO'].mean()
consumo_medio_por_categoria

Consumo total das fraudes por categoria

In [ ]:
consumo_total_por_categoria = fraudes_confirmadas.groupby('CATEGORIA')['CONS_MEDIDO'].sum()
consumo_total_por_categoria

Com base nas exceções, a determinação do consumo medido por número de fraudes.

In [ ]:
tabela_fraudes_excecao = fraudes_confirmadas.groupby('EXCECAO').agg({
    'CONS_MEDIDO': 'sum',
    'MATRICULA': 'count'  # Contando o número de fraudes
}).rename(columns={'MATRICULA': 'Número de Fraudes', 'CONS_MEDIDO': 'Consumo Total Medido'})

tabela_fraudes_excecao

### Análises e gráficos sazonais


1. **Consumo médio mensal:** Analisamos o consumo médio mensal de água para fraudes confirmadas e não confirmadas em 2023.
2. **Consumo por dia da semana:** Investigamos o consumo médio de água por dia da semana para identificar possíveis padrões sazonais.

Consumo médio mensal de água (registros não fraudulentos)

In [ ]:
fraudes_consumo_2023['REFERENCIA'] = pd.to_datetime(fraudes_consumo_2023['REFERENCIA'], errors='coerce')

# Filtrar os dados para o ano de 2023
dados_2023 = fraudes_consumo_2023[fraudes_consumo_2023['REFERENCIA'].dt.year == 2023]

# Separar fraudes confirmadas e não confirmadas
fraudes_confirmadas_2023 = dados_2023[dados_2023['fraude'] == 1]
fraudes_nao_confirmadas_2023 = dados_2023[dados_2023['fraude'] == 0]

# Adicionar a coluna de mês
fraudes_confirmadas_2023['MES'] = fraudes_confirmadas_2023['REFERENCIA'].dt.month
fraudes_nao_confirmadas_2023['MES'] = fraudes_nao_confirmadas_2023['REFERENCIA'].dt.month

# Consumo médio mensal para fraudes confirmadas
consumo_medio_confirmadas = fraudes_confirmadas_2023.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Consumo médio mensal para fraudes não confirmadas
consumo_medio_nao_confirmadas = fraudes_nao_confirmadas_2023.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Gráfico sazonal para fraudes confirmadas
fig_sazonal_confirmadas = px.line(consumo_medio_confirmadas, x='MES', y='CONS_MEDIDO',
                                  title='Consumo médio mensal de água em 2023 - Fraudes Confirmadas',
                                  labels={'MES': 'Mês', 'CONS_MEDIDO': 'Consumo Médio (m³)'},
                                  markers=True)

# Gráfico sazonal para fraudes não confirmadas
fig_sazonal_nao_confirmadas = px.line(consumo_medio_nao_confirmadas, x='MES', y='CONS_MEDIDO',
                                      title='Consumo médio mensal de água em 2023 - Fraudes Não Confirmadas',
                                      labels={'MES': 'Mês', 'CONS_MEDIDO': 'Consumo Médio (m³)'},
                                      markers=True)

# Exibir os gráficos
fig_sazonal_confirmadas.show()
fig_sazonal_nao_confirmadas.show()

In [ ]:
dados_2023['DIA_SEMANA'] = dados_2023['REFERENCIA'].dt.day_name()
consumo_medio_semanal = dados_2023.groupby('DIA_SEMANA')['CONS_MEDIDO'].mean().reset_index()
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
consumo_medio_semanal['DIA_SEMANA'] = pd.Categorical(consumo_medio_semanal['DIA_SEMANA'], categories=dias_ordenados, ordered=True)
consumo_medio_semanal = consumo_medio_semanal.sort_values('DIA_SEMANA')
fig_semanal = px.line(consumo_medio_semanal, x='DIA_SEMANA', y='CONS_MEDIDO',
                      title='Consumo Médio por Dia da Semana em 2023',
                      labels={'DIA_SEMANA': 'Dia da Semana', 'CONS_MEDIDO': 'Consumo Médio (m³)'},
                      markers=True)
fig_semanal.show()

In [ ]:
fraudes_consumo_2023_fraudes = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 1]
fraudes_consumo_2023_fraudes['REFERENCIA'] = pd.to_datetime(fraudes_consumo_2023_fraudes['REFERENCIA'], errors='coerce')
consumo_diario_por_categoria_fraude = fraudes_consumo_2023_fraudes.groupby(['REFERENCIA', 'CATEGORIA'])['CONS_MEDIDO'].mean().reset_index()
fig_fraude = px.line(consumo_diario_por_categoria_fraude,
                     x='REFERENCIA',
                     y='CONS_MEDIDO',
                     color='CATEGORIA',
                     title='Média de consumo (m³) mensal por categoria das fraudes confirmadas',
                     labels={'REFERENCIA': 'Data', 'CONS_MEDIDO': 'Média de Consumo Medido (m³)', 'CATEGORIA': 'Categoria'},
                     markers=True)

# Ajustar o layout para melhorar a visualização
fig_fraude.update_layout(xaxis_title='Data',
                         yaxis_title='Média de Consumo Medido (m³)')

# Exibir o gráfico
fig_fraude.show()

In [ ]:
fraudes_consumo_2023_sem_fraude = fraudes_consumo_2023[fraudes_consumo_2023['fraude'] == 0]
fraudes_consumo_2023_sem_fraude['REFERENCIA'] = pd.to_datetime(fraudes_consumo_2023_sem_fraude['REFERENCIA'], errors='coerce')
consumo_diario_por_categoria_sem_fraude = fraudes_consumo_2023_sem_fraude.groupby(['REFERENCIA', 'CATEGORIA'])['CONS_MEDIDO'].mean().reset_index()
fig_sem_fraude = px.line(consumo_diario_por_categoria_sem_fraude,
                         x='REFERENCIA',
                         y='CONS_MEDIDO',
                         color='CATEGORIA',
                         title='Média de consumo (m³) mensal por categoria não fraudulenta',
                         labels={'REFERENCIA': 'Data', 'CONS_MEDIDO': 'Média de Consumo Medido (m³)', 'CATEGORIA': 'Categoria'},
                         markers=True)
fig_sem_fraude.update_layout(xaxis_title='Data',
                             yaxis_title='Média de Consumo Medido (m³)')
fig_sem_fraude.show()

In [ ]:
consumo_mensal = fraudes_consumo_2023.groupby('ANO_MES')['CONS_MEDIDO'].sum().reset_index()
fig = px.line(consumo_mensal, x='ANO_MES', y='CONS_MEDIDO', title='Consumo Total Mensal de Água em 2023', labels={'ANO_MES': 'Mês', 'CONS_MEDIDO': 'Consumo Total (m³)'})
fig.show()

In [ ]:
consumo_medio_por_categoria = fraudes_consumo_2023.groupby('CATEGORIA')['CONS_MEDIDO'].mean().reset_index()
consumo_medio_por_categoria = consumo_medio_por_categoria.sort_values(by='CONS_MEDIDO', ascending=False)
consumo_medio_por_categoria['Porcentagem_Acumulada'] = consumo_medio_por_categoria['CONS_MEDIDO'].cumsum() / consumo_medio_por_categoria['CONS_MEDIDO'].sum() * 100
fig = px.bar(consumo_medio_por_categoria,
             x='CATEGORIA',
             y='CONS_MEDIDO',
             title='Volume médio de agua consumido por categoria',
             labels={'CONS_MEDIDO': 'Volume Médio Consumido (m³)', 'CATEGORIA': 'Categoria'})
fig.add_trace(
    go.Scatter(
        x=consumo_medio_por_categoria['CATEGORIA'],
        y=consumo_medio_por_categoria['Porcentagem_Acumulada'],
        mode='lines+markers',
        name='Porcentagem Acumulada',
        yaxis='y2'
    )
)
fig.update_layout(
    yaxis2=dict(
        title='Porcentagem Acumulada (%)',
        overlaying='y',
        side='right',
        range=[0, 100]
    ),
    xaxis_title='Categoria',
    yaxis_title='Volume Médio Consumido (m³)',
    showlegend=True
)
fig.show()

In [ ]:
consumo_medio_por_categoria = fraudes_consumo_2023_fraudes.groupby('CATEGORIA')['CONS_MEDIDO'].mean().reset_index()
consumo_medio_por_categoria = consumo_medio_por_categoria.sort_values(by='CONS_MEDIDO', ascending=False)
consumo_medio_por_categoria['Porcentagem_Relativa'] = (consumo_medio_por_categoria['CONS_MEDIDO'] / consumo_medio_por_categoria['CONS_MEDIDO'].sum()) * 100
fig = px.bar(consumo_medio_por_categoria,
             x='CATEGORIA',
             y='CONS_MEDIDO',
             title='Volume médio de água consumido por categoria em fraudes confirmadas',
             labels={'CONS_MEDIDO': 'Volume Médio Consumido (m³)', 'CATEGORIA': 'Categoria'})
fig.add_trace(
    go.Scatter(
        x=consumo_medio_por_categoria['CATEGORIA'],
        y=consumo_medio_por_categoria['Porcentagem_Relativa'],
        mode='lines+markers',
        name='Porcentagem Relativa (%)',
        yaxis='y2'
    )
)
fig.update_layout(
    yaxis2=dict(
        title='Porcentagem Relativa (%)',
        overlaying='y',
        side='right',
        range=[0, 100]
    ),
    xaxis_title='Categoria',
    yaxis_title='Volume Médio Consumido (m³)',
    showlegend=True
)
fig.show()

In [ ]:
fraudes_consumo_2023_fraudes['REFERENCIA'] = pd.to_datetime(fraudes_consumo_2023_fraudes['REFERENCIA'], errors='coerce')
contagem_fraudes_por_categoria = fraudes_consumo_2023_fraudes.groupby(['REFERENCIA', 'CATEGORIA']).size().reset_index(name='NUM_FRAUDES')

fig_fraudes = px.line(contagem_fraudes_por_categoria,
                      x='REFERENCIA',
                      y='NUM_FRAUDES',
                      color='CATEGORIA',
                      title='Número de Fraudes Diárias por Categoria',
                      labels={'REFERENCIA': 'Data', 'NUM_FRAUDES': 'Número de Fraudes', 'CATEGORIA': 'Categoria'},
                      markers=True)

# Ajustar o layout para melhorar a visualização
fig_fraudes.update_layout(xaxis_title='Data',
                          yaxis_title='Número de Fraudes')

fig_fraudes.show()

### Análises geoespaciais


Para entender a distribuição das fraudes e consumo em termos geográficos, geramos gráficos de dispersão e mapas com coordenadas de latitude e longitude, representando o volume de consumo e a quantidade de fraudes.


In [ ]:
# Agrupar por matrícula (ou outro identificador único) e somar o consumo medido
agrupado = fraudes_consumo_2023.groupby('MATRICULA').agg({
    'CONS_MEDIDO': 'sum',
    'COD_LATITUDE': 'first',
    'COD_LONGITUDE': 'first'
}).reset_index()

# Criar o gráfico geoespacial com os dados agregados
fig_geo_agrupado = px.scatter_mapbox(
    agrupado,
    lat='COD_LATITUDE',
    lon='COD_LONGITUDE',
    color='CONS_MEDIDO',
    size='CONS_MEDIDO',
    size_max=15,
    zoom=10,
    mapbox_style="carto-positron",
    title='Mapeamento Geoespacial de Consumo Medido Agregado em Casos de Fraude (Não-Residencial, 2023)',
    width=1000,
    height=1000
)

# Exibir o gráfico
fig_geo_agrupado.show()

In [ ]:
# Filtrar o DataFrame para incluir apenas fraudes e categorias não residenciais
fraudes_consumo_2023_fraudes = fraudes_consumo_2023[
    (fraudes_consumo_2023['fraude'] == 1) &
    (fraudes_consumo_2023['CATEGORIA'] != 'RESIDENCIAL')
]
# Remover linhas com valores nulos em 'COD_LATITUDE' e 'COD_LONGITUDE'
fraudes_consumo_2023_fraudes_clean = fraudes_consumo_2023_fraudes.dropna(subset=['COD_LATITUDE', 'COD_LONGITUDE'])

# Contar o número de fraudes por coordenada (latitude e longitude), matrícula e categoria
fraudes_contagem = fraudes_consumo_2023_fraudes_clean.groupby(['COD_LATITUDE', 'COD_LONGITUDE', 'MATRICULA', 'CATEGORIA']).size().reset_index(name='NUM_FRAUDES')

# Criar o gráfico geoespacial com tamanho aumentado e inclusão de matrícula e categoria
fig_geo = px.scatter_mapbox(
    fraudes_contagem,
    lat='COD_LATITUDE',
    lon='COD_LONGITUDE',
    color='NUM_FRAUDES',  # Usar a quantidade de fraudes para colorir os pontos
    size='NUM_FRAUDES',  # Usar a quantidade de fraudes para dimensionar os pontos
    size_max=25,  # Aumentar o tamanho máximo dos pontos
    hover_name='MATRICULA',  # Adicionar matrícula como texto ao passar o cursor
    hover_data={'CATEGORIA': True, 'NUM_FRAUDES': True},  # Incluir a categoria e número de fraudes no hover
    zoom=10,
    mapbox_style="carto-positron",
    title='Mapeamento Geoespacial de Quantidade de Fraudes (Não-Residencial, 2023)',
    width=1000,  # Aumentar a largura do gráfico
    height=1000  # Aumentar a altura do gráfico
)

# Exibir o gráfico
fig_geo.show()

In [ ]:
consumo_total_por_fraude = fraudes_confirmadas.groupby('MATRICULA')['CONS_MEDIDO'].sum().reset_index()
consumo_total_por_fraude['OUTLIER'] = (consumo_total_por_fraude['CONS_MEDIDO'] > consumo_total_por_fraude['CONS_MEDIDO'].mean() + 3*consumo_total_por_fraude['CONS_MEDIDO'].std())
consumo_total_por_fraude['OUTLIER'] = consumo_total_por_fraude['OUTLIER'].astype(int)
consumo_total_por_fraude

## Análise qualitativa dos resultados obtidos

A análise exploratória realizada até agora fornece uma base para entender os padrões de consumo de água e a incidência de fraudes em diferentes categorias e regiões. Com os insights obtidos, podemos começar a identificar variáveis e características importantes que poderão alimentar o modelo de deep learning para calcular a probabilidade de fraude. A seguir, estão dispostos os principais pontos de cada parte da análise.

### **1. Estrutura e qualidade dos dados**
* Dimensões dos dados: A análise inicial das dimensões dos dados revelou a presença de um grande número de registros, o que é positivo para a robustez do modelo a ser construído, especialmente considerando a existência das bases dos anos anteriores. A verificação dos tipos de dados e a presença de valores nulos indicam que os dados estão em um estado relativamente bom, com algumas colunas que podem requerer limpeza adicional antes da modelagem.

* Valores Únicos: A análise dos valores únicos em colunas como EXCECAO, DSC_SIMULTANEA, CATEGORIA e DSC_OCORRENCIA sugere que essas colunas contêm informações que podem ser cruciais para identificar padrões suspeitos. Por exemplo, a presença de exceções e retenções pode indicar irregularidades que precisam ser mais investigadas.

### **2. Integração dos dados de consumo e fraudes**

* Unificação dos dados: A integração dos dados de consumo com os registros de fraudes para o ano de 2023 permitiu a criação de um conjunto de dados consolidado, onde cada matrícula é identificada como fraudulenta ou não. Este passo é essencial para criar uma variável de resposta clara (fraude), que será a variável alvo no modelo construído.

* Conversão e manipulação de datas: A padronização das datas foi um passo importante para permitir análises temporais, o que é fundamental para capturar tendências e sazonalidades que podem influenciar o comportamento de fraude.

### **3. Análise de consumo e tarifas**
* Cálculo da fatura mensal: A implementação de uma função para calcular a fatura mensal com base em tarifas fixas e variáveis por subcategoria é um ponto forte. Isso não apenas ajuda a entender o impacto financeiro das fraudes, mas também pode revelar outliers em consumo que, por sua vez, podem estar associados a práticas fraudulentas.

* Impacto das tarifas: A análise das tarifas aplicadas em diferentes subcategorias pode ser explorada mais profundamente. Subcategorias com tarifas mais elevadas podem estar mais propensas a fraudes devido ao incentivo financeiro para reduzir custos.

### **4. Distribuição das Fraudes**
* Distribuição por subcategoria: As fraudes foram distribuídas de maneira desigual entre as diferentes subcategorias. Subcategorias com maior incidência de fraudes precisam de uma atenção especial. Esse insight pode guiar o foco das estratégias de mitigação e ajudar a afinar o modelo de predição, ajustando-o para capturar variações específicas por subcategoria.

* Reincidência em matrículas: A identificação de matrículas reincidentes em fraudes é um dado valioso. Esses clientes reincidentes podem ter características específicas que o modelo de deep learning pode aprender a identificar, aumentando a precisão na detecção de fraudes futuras.

### **5. Análises Temporais**
* Sazonalidade e consumo médio mensal: A análise sazonal revelou que o consumo médio varia ao longo do ano, tanto para fraudes confirmadas quanto para registros não fraudulentos. Essa variação sazonal pode ser uma característica importante para o modelo de predição, sugerindo que a incorporação de variáveis temporais pode melhorar o desempenho do modelo.

* Consumo por dia da semana: A diferença no consumo médio de água por dia da semana indica que padrões de comportamento diário também podem influenciar a probabilidade de fraude. Este insight deve ser incorporado ao modelo para capturar essas variações comportamentais.

### **6. Análises geoespaciais**
* Distribuição geográfica das fraudes: Os gráficos geoespaciais e a análise de clusters mostraram que as fraudes não estão distribuídas uniformemente no espaço. Certas regiões apresentam uma maior concentração de fraudes, o que pode ser influenciado por fatores socioeconômicos ou culturais. Incorporar dados geográficos no modelo de predição pode aumentar significativamente sua acurácia.

### **7. Análise por Categoria**
* Consumo médio e total por categoria: A análise de consumo por categoria mostrou que certas categorias têm um consumo médio significativamente maior, o que pode indicar práticas fraudulentas sofisticadas. Essas categorias devem ser monitoradas de perto, e o modelo deve ser treinado para identificar anomalias dentro dessas faixas de consumo.

* Impacto acumulado: A porcentagem acumulada de consumo pode ajudar a identificar outliers que estão contribuindo desproporcionalmente para o consumo total, o que é um indicador clássico de possível fraude.

### **8. Outliers e anomalias**
* Identificação de outliers: A detecção de outliers no consumo, especialmente em matrículas com histórico de fraude, pode ser um dos fatores determinantes para o modelo. Esses outliers devem ser tratados com cuidado na modelagem, pois podem influenciar a decisão do modelo de forma significativa.

### **9. Conclusão e próximos passos**
A análise exploratória realizada até aqui forneceu uma compreensão abrangente dos dados de consumo e fraudes, identificando padrões importantes e características que poderão ser usadas na construção de um modelo de deep learning para predição de fraudes. A inclusão de variáveis temporais, geográficas, socioeconômicas, e de consumo, bem como a identificação de outliers e padrões sazonais, são fundamentais para melhorar a acurácia e a robustez do modelo.

Os próximos passos devem focar em:

* Feature engineering: Criar novas variáveis baseadas nos insights obtidos, como índices de sazonalidade, distâncias geográficas entre fraudes, e agregações temporais.

* Modelagem preliminar: Testar modelos básicos de classificação (como Random Forest ou Gradient Boosting) para identificar quais características são mais importantes (ainda estamos estudando se iremos, de fato, realizar esse passo, mas, por ora, parece ser uma alternativa palpável).

* Desenvolvimento do modelo de deep learning: Com base nas variáveis identificadas, treinar um modelo de deep learning robusto, ajustando parâmetros e arquitetura para maximizar a performance preditiva.

Com esses passos, será possível construir um modelo capaz de determinar a probabilidade de fraudes com alta precisão, contribuindo significativamente para a redução de perdas financeiras e aprimorando as estratégias de prevenção de fraudes da Aegea.